In [1]:
import ROOT
import sys
import numpy as np
from dataclasses import dataclass
from utils import *

%jsroot on

Welcome to JupyROOT 6.28/00


# Defining Functions

## Getting Hists

Using pyroot to get the detector map and generally a list of histograms from a `ROOT` file measureing 

In [5]:
def get_det_angle (file_name: str) -> list[int]:
    """Takes a string of a root file and returns a list containing the detector map"""

    file = ROOT.TFile(file_name)
    det_pos_graph = file.Get("det_pos_graph")
    det_pos = []
    pos_angle = [-1,71,90,109,109,90,71,90,-1,71,90,109,109,90,71,90,-1,36,72,108,144,-1,-1,-1,-1,144,108,72,36,-1,-1,-1] 
    det_angle = []

    for i in range(det_pos_graph.GetN()):
        det_pos.append(int(det_pos_graph.Eval(i)))

    file.Close()
    
    for i in range(len(det_pos)):
        det_angle.append(pos_angle[det_pos[i]])
    det_angle.append(-1)
    return det_angle

def get_hist (file_name: str, dir_name: str) -> list[ROOT.TH1]:
    """Takes a string of a root file and a name of a histogram and returns a list of histograms"""

    list_of_hist = []
    file = ROOT.TFile(file_name)

    dir=file.Get(dir_name)

    for i, key in enumerate(dir.GetListOfKeys()):
        list_of_hist.append(dir.Get(key.GetName()))
        list_of_hist[i].SetDirectory(0)

    file.Close()
    
    return list_of_hist

In [6]:
# def get_hist_uproot(file_name: str, dir_name: str) -> list[ROOT.TH1]:

In [7]:
hist = get_hist("stage0_output_NaI_1.root","hEn")

nbins = hist[1].GetXaxis().GetNbins()

bin_edges = [hist[1].GetXaxis().GetBinLowEdge(bin) for bin in range(1, nbins+1)]

# bin_edges

## Calculate Subtracted Histogram

First the scale is caclculated from the background and used to do the background subtraction

In [8]:
def calc_scale(hEgam_bkg: list[ROOT.TH1], gate: list[float], gate_bkg: list[float]) -> list[float]:
    """Takes a list of histograms, calculates the scale and return the scales as a list"""

    scale = []

    for i in range(len(hEgam_bkg)):
        try:     
            num = hEgam_bkg[i].Integral(hEgam_bkg[i].GetXaxis().FindBin(gate[0]),hEgam_bkg[i].GetXaxis().FindBin(gate[1]))
            dem = hEgam_bkg[i].Integral(hEgam_bkg[i].GetXaxis().FindBin(gate_bkg[0]),hEgam_bkg[i].GetXaxis().FindBin(gate_bkg[1]))
            scale.append(num/dem)
        except ZeroDivisionError:
            scale.append(0)
    
    return scale

def calc_hEn_sub(hist: list, hist_bkg: list, scale: list[float]) -> list[ROOT.TH1]:
    """Takes 2 lists of histograms and a list of scales and calculates a new histogram then returns the histograms as a list"""
        
    assert len(hist) == len(hist_bkg)
    
    hist_bkg_sub = []
    for i in range(len(hist)):
        hist_bkg_sub.append(hist[i]-hist_bkg[i]*scale[i])
        hist_bkg_sub[i].SetName(hist[i].GetName()+"_bkgsub")
        hist_bkg_sub[i].SetDirectory(0)

    return hist_bkg_sub

def calc_hEgam_bkg ( list_of_hist: list[ROOT.TH1], x1: float, x2: float) -> list[ROOT.TH1]:
    """Takes a list of histograms and a range in the form of x1 and x2 and returns a list of histograms"""

    hEgam_bkg = []
    for i, hist in enumerate(list_of_hist):
        hist.GetXaxis().SetRangeUser(x1,x2)
        hEgam_bkg.append(hist.ShowBackground(50))
        hEgam_bkg[i].SetName(hist.GetName()+"_bkg")

    return hEgam_bkg

## Calculating N_LH

In [9]:
def calc_N_LH(hist: list[ROOT.TH1], energy: float, width: float) -> tuple[np.array, np.array]:
    """Takes a list of histograms and returns a list of N_L, N_H for each histogram"""
    
    N_L = []
    N_H = []
    

    for i in range(len(hist)):
        energy_bin = hist[i].FindBin(energy)
        width_bin = int(width//hist[i].GetBinWidth(energy_bin))
        N_L.append(hist[i].Integral(energy_bin - width_bin, energy_bin - 1))
        N_H.append(hist[i].Integral(energy_bin, energy_bin + width_bin))
    
        

    return np.array(N_L), np.array(N_H)

In [10]:
# calc_N_LH(get_hist("stage0_output_NaI_1.root","hEn"),13.6,0.5)

## Adding by Angle

In [11]:
def add_by_angle(list_by_det: list[list[any]], det_angle: list[list[int]]) -> list[any]:
    """DEPRICATED
    
    Takes a lists of lists to be added by detector
    """
    
    angle_list = [36,71,72,90,108,109,144]
    list_by_angle = [0] * len(angle_list)
    
    for i, sublist in enumerate(list_by_det):
        for j, value in enumerate(sublist):
             for k, angle in enumerate(angle_list):
                if angle == det_angle[i][j]:
                    list_by_angle[k]+=value
               
    return list_by_angle

In [12]:
def sort_by_angle(list_by_det: list[any], det_angle_map: list[int]) -> list[any]:
    """Takes a lists and a detector angle map and returns an angle sorted list """
    
    assert len(list_by_det) == len(det_angle_map)
                  
    angle_list = [36,71,72,90,108,109,144]
    list_by_angle = [0] * len(angle_list)
    
    for i, value in enumerate(list_by_det):
         for j, angle in enumerate(angle_list):
            if angle == det_angle_map[i]:
                list_by_angle[j]+=value
               
    return np.array(list_by_angle)

In [13]:
# list1 = [[0.01,0.01,0.01],[0.1,0.1,0.1],[1,1,1]]
# list2 = [[36,109,144],[36,109,144],[36,109,144]]
# add_by_angle(list1,list2)
# sort_by_angle([1,2,3],[36,109,108,90])

## Calculating A_LH

In [14]:
def A_LH (N_L: float, N_H: float) -> float:
    """Takes two floats N_L and N_H and returns A_LH"""

    return (N_L - N_H) / (N_L + N_H)

def dA_LH (N_L: float, N_H: float) -> float:
    """Takes two floats N_L and N_H and returns dA_LH"""

    return 2 * np.sqrt((N_L * N_L * N_H + N_L * N_H * N_H)) / (N_L + N_H) / (N_L + N_H)

In [15]:
# arr1 = np.array([1, 2, 3, -1, 5])
# arr2 = np.array([2, 3, 4, 1, 6])
# A_LH(arr1,arr2)

## Drawing Histograms

In [16]:
def draw_hist (list_of_hist: list[ROOT.TH1]) -> list[ROOT.TCanvas]:
    """Takes a list of histograms and draws each one"""

    if type(list_of_hist) is not list: list_of_hist = [list_of_hist]
    
    hist_canvas = []
    
    for i, hist in enumerate(list_of_hist): 
        if hist.GetEntries() != 0:
            hist_canvas.append(ROOT.TCanvas(hist.GetName(),hist.GetName(),500,400))
            hist.Draw("E1")
    for canvas in hist_canvas:     
        canvas.Draw()
    return hist_canvas

In [17]:
# hist = get_hist("stage0_output_NaI_1.root","hEn")
# draw_hist(hist[1])

In [18]:
def create_A_LH_graph(x: np.array, y: np.array, dx: np.array = None , dy: np.array = None, title: str = "A_{LH}") -> ROOT.TGraphErrors:
    """Takes a np.array of the x and y values and their errors and returns a TGraphErrors"""
    
    if dx is None: dx = np.zeros_like(x)
    if dy is None: dy = np.zeros_like(y)

    x = x.astype('f')
    y = y.astype('f')
    dx = dx.astype('f')
    dy = dy.astype('f')

    global graph
    graph = ROOT.TGraphErrors(len(x), x, y, dx, dy)

    graph.SetMarkerStyle(20)
    graph.GetXaxis().SetTitle("cos#theta_{#gamma}")
    graph.GetYaxis().SetTitle("A_{LH}")
    graph.SetTitle(title)

    return graph

def linear_fit_and_plot (graph: ROOT.TGraphErrors) -> ROOT.TCanvas:
    """Takes TGraphErrors returns a plot with a linear fit"""

    graph_canvas = ROOT.TCanvas(graph.GetTitle(), graph.GetTitle(), 600,500)
    graph.Draw("AP")
    fitFunc = ROOT.TF1("fitFunc", "[0] * x + [1]")
    graph.Fit(fitFunc, "Q")
    ROOT.gStyle.SetOptFit(1)
    graph_canvas.Draw()

    return graph_canvas

In [19]:
# g=create_A_LH_graph(np.array([0,1,2,3]), np.array([0,1,2,3]),np.array([0,1,2,3]),np.array([0,1,2,3]))
# linear_fit_and_plot(g)

## Converting TOF to En

In [20]:
def htof_to_hEn (list_of_hist: ROOT.TH1, length: float) -> ROOT.TH1:
    """Takes a list of histograms and converts from time-of-flight to neutron energy"""

    y = list_of_hist[1].GetArray()
    y.SetSize(list_of_hist[1].GetNbinsX())
    y = np.array(y)

    return y

In [33]:
# hist = get_hist("stage0_output_NaI_1.root","hTOF")
# htof_to_hEn(hist, 21.5)

## RDataFrame Testing

In [32]:
# ROOT.EnableImplicitMT()
# tree_name = "rawTree"
# file_name = "/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_001[4-9]*"

# df = ROOT.RDataFrame(tree_name, file_name)
# print(df.GetColumnNames())
# print(df.GetColumnType("tof"))
# df = df.Define("En","pow((72.3*21.5/(tof*10./1000.0)),2)")

# print(df.GetDefinedColumnNames())

# print(df.Display())


# Testing Script

## Defining Globals

In [34]:
E_p = 22.2
G_p = 0.5
hEgam_gate_FA = [6637,6702] # 127I
hEgam_gate_FE = [6150,6194] # 127I
hEgam_gate_FA_bkg = [6777,7044] # 127I

angle = np.array([36,71,72,90,108,109,144])
cos_angle = np.cos(np.pi/180*angle)

input_file1 = "stage0_output_NaI_1.root"
input_file2 = "stage0_output_NaI_2.root"

## Getting Histograms

In [35]:
det_angle1 = get_det_angle(input_file1)
hEn1 = get_hist(input_file1,"hEn_gated_FA")
hEn_bkg1 = get_hist(input_file1,"hEn_gated_FA_bkg")
hEgam1 = get_hist(input_file1,"hEgam")

det_angle2 = get_det_angle(input_file2)
hEn2 = get_hist(input_file2,"hEn_gated_FA")
hEn_bkg2 = get_hist(input_file2,"hEn_gated_FA_bkg")
hEgam2 = get_hist(input_file2,"hEgam")

## Performing Background Subtractions

In [36]:
hEgam_bkg2 = calc_hEgam_bkg(hEgam2,6500,7100)
scale2 = calc_scale(hEgam_bkg2, hEgam_gate_FA, hEgam_gate_FA_bkg)
hEn_sub2 = calc_hEn_sub(hEn2, hEn_bkg2, scale2)


hEgam_bkg1 = calc_hEgam_bkg(hEgam1,6500,7100)
scale1 = calc_scale(hEgam_bkg1, hEgam_gate_FA, hEgam_gate_FA_bkg)
hEn_sub1 = calc_hEn_sub(hEn1, hEn_bkg1, scale1)

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


## Calculating N_L and N_H

In [37]:
N_L_det2, N_H_det2 = calc_N_LH(hEn_sub2, E_p, G_p)
N_L_det1, N_H_det1 = calc_N_LH(hEn_sub1, E_p, G_p)

## Adding N_L and N_H By Angle

In [38]:
N_L_angle1 = sort_by_angle(N_L_det1,det_angle1)
N_H_angle1 = sort_by_angle(N_H_det1,det_angle1)
N_L_angle2 = sort_by_angle(N_L_det2,det_angle2)
N_H_angle2 = sort_by_angle(N_H_det2,det_angle2)

In [39]:
# N_L_angle1

## Adding Runs

In [40]:
N_L_angle = N_L_angle1 + N_L_angle2 
N_H_angle = N_H_angle1 + N_H_angle2 


In [41]:
# print(N_L_angle)
# print(N_H_angle)
# N_L_angle

## Calculating A_LH and dA_LH

In [42]:
A_LH_angle = A_LH(N_L_angle,N_H_angle)
dA_LH_angle = dA_LH(N_L_angle,N_H_angle)

A_LH_angle1 = A_LH(N_L_angle1,N_H_angle1)
dA_LH_angle1 = dA_LH(N_L_angle1,N_H_angle1)

A_LH_angle2 = A_LH(N_L_angle2,N_H_angle2)
dA_LH_angle2 = dA_LH(N_L_angle2,N_H_angle2)


/var/folders/xg/rnpq_36x4jqgm4p1d_0gd47c0000gn/T/ipykernel_97466/870674217.py:9: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.sqrt((N_L * N_L * N_H + N_L * N_H * N_H)) / (N_L + N_H) / (N_L + N_H)


In [44]:
# graph = ROOT.TGraphErrors(len(A_LH_angle[1:]),cos_angle[1:], A_LH_angle[1:], np.zeros_like(A_LH_angle[1:]), dA_LH_angle[1:])
# graph.SetMarkerStyle(20)
# graph.GetXaxis().SetTitle("cos#theta_{#gamma}")
# graph.GetYaxis().SetTitle("A_{LH}")
# # graph.SetTitle(title)

# canvas = ROOT.TCanvas("canvas", "My Graph", 600, 500)
# graph.Draw()
# fitFunc = ROOT.TF1("fitFunc", "[0]*x + [1]")
# graph.Fit(fitFunc, "Q")
# ROOT.gStyle.SetOptFit(1)
# graph.Draw("AP")
# canvas.Draw()
A_LH_graph = create_A_LH_graph(cos_angle, A_LH_angle, dy = dA_LH_angle)
linear_fit_and_plot(graph)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: A_{LH}
